In [15]:
import requests
import pandas as pd
import time
from random import randint, choice

In [16]:
URL_API = "https://graphql-gateway.axieinfinity.com/graphql"

headers = {
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-ES,es;q=0.9,en;q=0.8',
    'cache-control': 'no-cache',
    'content-length': '1010',
    'content-type': 'application/json',
    'cookie': '_ga=GA1.2.581236636.1631802371; _gid=GA1.2.1060654813.1631802371; cf_clearance=ps3wfb304R6sKsauq5bsfRJh.xrDvMo2TWlLub0Vf2s-1631892833-0-250; _gat_gtag_UA_150383258_1=1',
    'origin': 'https://marketplace.axieinfinity.com',
    'pragma': 'no-cache',
    'referer': 'https://marketplace.axieinfinity.com/',
    'sec-ch-ua': '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
}

In [17]:
query = """query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String) {
  axies(auctionType: $auctionType, criteria: $criteria, from: $from, sort: $sort, size: $size, owner: $owner) {
    total
    results {
      ...AxieDetail
      transferHistory(from: $from, size: $size) {
        ...TransferRecords
        __typename
      }
      ethereumTransferHistory(from: $from, size: $size) {
        ...TransferRecords
        __typename
      }   
      __typename
    }
    __typename
  }
}

fragment AxieDetail on Axie {
  id
  image
  class
  chain
  name
  birthDate
  bodyShape
  sireId
  sireClass
  matronId
  matronClass
  stage
  title
  breedCount
  level
  stats {
    ...AxieStats
    __typename
  }
  __typename
}

fragment AxieStats on AxieStats {
  hp
  speed
  skill
  morale
  __typename
}

fragment TransferRecords on TransferRecords {
  total
  results {
    from
    to
    timestamp
    txHash
    withPrice
    __typename
  }
  __typename
}
"""

In [18]:
payload = {
    "operationName": "GetAxieBriefList",
    "variables": {
        "from": 0,
        "size": 10,
        "sort": "Latest",
        "auctionType": "Sale",
        "owner": None,
        "criteria": {
            "region": None,
            "parts": None,
            "bodyShapes": None,
            "classes": None,
            "stages": [
                4
            ],
            "numMystic": None,
            "pureness": None,
            "title": None,
            "breedable": None,
            "breedCount": None,
            "hp": [],
            "skill": [],
            "speed": [],
            "morale": []
        }
    },
    "query": query
}

In [19]:
def get_random_ua():
    with open('agents.txt', encoding='utf-8') as f:
        lines = f.readlines()
    return  choice(lines).replace('\n','')

In [ ]:
TOTAL = requests.post(URL_API, json=payload).json()['data']['axies']['total']

START = 0
SIZE = 75
NO_REQUESTS = TOTAL // SIZE
CHECKPOINTS = SIZE * 100 # Guarda un Checkpoint cada 5000 requests
col_dtypes = {
    'id':'uint32',
    'sireId':'uint16',
    'matronId':'uint16',
    'birthDate':'uint32',
    'stage':'uint8',
    'breedCount':'uint8',
    'level':'uint8',
    'hp':'uint8',
    'speed':'uint8',
    'skill':'uint8',
    'morale':'uint8',
    'withPrice': 'float32'
}

payload['variables']['size'] = SIZE

df = pd.DataFrame()

for i in range(0, NO_REQUESTS * SIZE, SIZE):
  
    if i % (10 * SIZE ) == 0: print('--> Getting from', i, 'To', i + SIZE);

    payload['variables']['from'] = i
    headers['user-agent'] = get_random_ua()
    try:
        time.sleep(randint(3,15))
        res = requests.post(URL_API, json=payload, headers=headers)
        if res.status_code == 200:
            axies = res.json()['data']['axies']['results']
        else:
            raise 'Invalid code'

    except:
        print('------> Error Batch #', i, '-', i+SIZE)
        axies = []

    transactions = []
    for axie in axies:
        stats = axie.pop('stats')
        axie = { **axie, **stats}
        txns = axie.pop('ethereumTransferHistory')['results'] + axie.pop('transferHistory')['results']
        transactions += [ {**axie, **t} for t in txns ]

    try:
        transactions = pd.DataFrame(transactions).astype(col_dtypes).drop(columns='__typename')
        transactions.withPrice = transactions.withPrice / 1e18
        df = df.append(transactions)
    
    except:
        print('------> Empty Batch #', i, '-', i+SIZE)

    if i and ( i + SIZE ) % CHECKPOINTS == 0 and not df.empty:
        print('---------> Saving checkpoint!')
        df = df.set_index('txHash')
        df.to_csv(f'./dataset/dataset_{ i + SIZE - CHECKPOINTS }_{ i + SIZE }.csv')
        df = pd.DataFrame()

print('---------> Saving last checkpoint!')
if not df.empty: df = df.set_index('txHash');
df.to_csv(f'./dataset/dataset_last.csv')

--> Getting from 0 To 75
------> Error Batch # 0 - 75
------> Empty Batch # 0 - 75
--> Getting from 750 To 825
--> Getting from 1500 To 1575
--> Getting from 2250 To 2325
--> Getting from 3000 To 3075
--> Getting from 3750 To 3825
--> Getting from 4500 To 4575
--> Getting from 5250 To 5325
--> Getting from 6000 To 6075
--> Getting from 6750 To 6825
------> Error Batch # 7275 - 7350
------> Empty Batch # 7275 - 7350
---------> Saving checkpoint!
--> Getting from 7500 To 7575
--> Getting from 8250 To 8325
--> Getting from 9000 To 9075
--> Getting from 9750 To 9825
------> Empty Batch # 9975 - 10050
------> Empty Batch # 10050 - 10125
------> Empty Batch # 10125 - 10200
------> Empty Batch # 10200 - 10275
------> Empty Batch # 10275 - 10350
------> Empty Batch # 10350 - 10425
------> Empty Batch # 10425 - 10500
--> Getting from 10500 To 10575
------> Empty Batch # 10500 - 10575
------> Empty Batch # 10575 - 10650
------> Empty Batch # 10650 - 10725
------> Empty Batch # 10725 - 10800
----

------> Empty Batch # 24450 - 24525
------> Empty Batch # 24525 - 24600
------> Empty Batch # 24600 - 24675
------> Empty Batch # 24675 - 24750
--> Getting from 24750 To 24825
------> Empty Batch # 24750 - 24825
------> Empty Batch # 24825 - 24900
------> Empty Batch # 24900 - 24975
------> Empty Batch # 24975 - 25050
------> Empty Batch # 25050 - 25125
------> Empty Batch # 25125 - 25200
------> Empty Batch # 25200 - 25275
------> Empty Batch # 25275 - 25350
------> Empty Batch # 25350 - 25425
------> Empty Batch # 25425 - 25500
--> Getting from 25500 To 25575
------> Empty Batch # 25500 - 25575
------> Empty Batch # 25575 - 25650
------> Empty Batch # 25650 - 25725
------> Empty Batch # 25725 - 25800
------> Empty Batch # 25800 - 25875
------> Empty Batch # 25875 - 25950
------> Empty Batch # 25950 - 26025
------> Empty Batch # 26025 - 26100
------> Empty Batch # 26100 - 26175
------> Empty Batch # 26175 - 26250
--> Getting from 26250 To 26325
------> Empty Batch # 26250 - 26325
----